In [ ]:
import pandas as pd

# -------------------------------
# Config / Mappings
# -------------------------------
RESERVED_MAPPING = {
    "not reserved": "no",
    "vanilla": "yes",
    None: "no"
}


TRANSLATIONS = {
    # finish column
    "UNGEBEIZT": "UNPICKLED",
    "GEBEIZT": "PICKLED",
    "GEBEIZT UND GEGLÜHT": "PICKLED AND ANNEALED",
    #
    # description column (defects / notes)
    "LÄNGS- ODER QUERISSE": "LONGITUDINAL OR TRANSVERSE CRACKS",
    "KANTENFEHLER - FS-KANTENRISSE": "EDGE DEFECT - FS EDGE CRACKS",
    "SOLLASS (GEWICHT) UNTERSCHRITTEN": "TARGET WEIGHT UNDERACHIEVED"
}

STRING_COLUMNS = ["quality", "finish", "description", "grade", "reserved"]
NUMERIC_COLUMNS = ["thickness_mm", "width_mm", "weight_kg", "quantity", "rp02", "rm", "ag", "ai"]

COLUMN_ORDER = ["article_id", "grade", "description", "finish", "thickness_mm",
                "width_mm", "weight_kg", "quantity", "rp02", "rm", "ag", "ai", "reserved"]

# -------------------------------
# STEP 1: Helper functions
# -------------------------------
def normalize_strings(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(str).str.upper().str.strip()
    return df

def normalize_numeric(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for col in columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(float)
    return df

def apply_translations(df: pd.DataFrame, columns: list[str], translations: dict) -> pd.DataFrame:
    for col in columns:
        if col in df.columns:
            df[col] = df[col].replace(translations)
    return df

def normalize_reserved(df: pd.DataFrame, col: str = "reserved") -> pd.DataFrame:
    if col in df.columns:
        df[col] = df[col].map(RESERVED_MAPPING).fillna("no")
    return df

# -------------------------------
# STEP 2: Load Data
# -------------------------------
df1 = pd.read_excel("supplier_data1.xlsx")
df2 = pd.read_excel("supplier_data2.xlsx")

# -------------------------------
# STEP 3: Rename Columns
# -------------------------------
df1 = df1.rename(columns={
    "Quality/Choice": "quality",
    "Grade": "grade",
    "Finish": "finish",
    "Thickness (mm)": "thickness_mm",
    "Width (mm)": "width_mm",
    "Description": "description",
    "Gross weight (kg)": "weight_kg",
    "Quantity": "quantity",
    "RP02": "rp02",
    "RM": "rm",
    "AG": "ag",
    "AI": "ai"
})

df2 = df2.rename(columns={
    "Material": "grade",
    "Description": "finish",
    "Article ID": "article_id",
    "Weight (kg)": "weight_kg",
    "Quantity": "quantity",
    "Reserved": "reserved"
})
df2["description"] = df2["finish"]

# -------------------------------
# STEP 4: Standardize values
# -------------------------------
df1 = normalize_strings(df1, STRING_COLUMNS)
df2 = normalize_strings(df2, STRING_COLUMNS)

df1 = normalize_numeric(df1, NUMERIC_COLUMNS)
df2 = normalize_numeric(df2, NUMERIC_COLUMNS)

df1 = normalize_reserved(df1)
df2 = normalize_reserved(df2)

# -------------------------------
# STEP 5: Concatenate datasets
# -------------------------------
inventory_dataset = pd.concat([df1, df2], ignore_index=True)

# Fill missing essential columns
inventory_dataset["reserved"] = inventory_dataset["reserved"].fillna("no")
inventory_dataset["article_id"] = inventory_dataset["article_id"].fillna("not available").astype(str).str.upper().str.strip()
inventory_dataset = apply_translations(inventory_dataset, ["finish", "description"], TRANSLATIONS)
inventory_dataset.fillna({
    "quality": "UNKNOWN",
    "finish": "UNKNOWN",
    "description": "NO DEFECTS",
    "grade": "UNKNOWN",
    "thickness_mm": 0.0,
    "width_mm": 0.0,
    "weight_kg": 0.0,
    "quantity": 0,
    "rp02": 0.0,
    "rm": 0.0,
    "ag": 0.0,
    "ai": 0.0
}, inplace=True)
inventory_dataset = inventory_dataset.reindex(columns=COLUMN_ORDER)

# -------------------------------
# STEP 6: Save result
# -------------------------------
inventory_dataset.to_csv("inventory_dataset.csv", index=False)
print("✅ inventory_dataset.csv has been created successfully!")


✅ inventory_dataset.csv has been created successfully!


In [15]:
inventory_dataset

,article_id,grade,description,finish,thickness_mm,width_mm,weight_kg,quantity,rp02,rm,ag,ai,reserved
0,NOT AVAILABLE,C200S,LONGITUDINAL OR TRANSVERSE CRACKS,PICKLED AND ANNEALED,2.77,1100.0,13983.0,0.00,333.6,606.2,16.11,0.0054,no
1,NOT AVAILABLE,C300S,LONGITUDINAL OR TRANSVERSE CRACKS,UNPICKLED,2.65,1075.0,13047.0,0.00,717.7,0.0,16.11,0.0046,no
2,NOT AVAILABLE,C100S,EDGE DEFECT - FS EDGE CRACKS,PICKLED AND ANNEALED,2.20,1100.0,14155.0,10.84,368.9,0.0,0.00,0.0061,no
3,NOT AVAILABLE,C100S,LONGITUDINAL OR TRANSVERSE CRACKS,PICKLED,2.86,1100.0,11381.0,22.87,368.9,601.7,0.00,0.0062,no
4,NOT AVAILABLE,C300S,SOLLMASSE (GEWICHT) UNTERSCHRITTEN,UNPICKLED,2.88,1050.0,10072.0,22.87,0.0,1213.0,0.00,0.0041,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,23047543.0,DX51D +Z140,MATERIAL IS NOT OILED,MATERIAL IS NOT OILED,0.00,0.0,9940.0,96.00,0.0,0.0,0.00,0.0000,no
96,23045292.0,DX51D +AZ150,MATERIAL IS PAINTED,MATERIAL IS PAINTED,0.00,0.0,23884.0,51.00,0.0,0.0,0.00,0.0000,no
97,23047408.0,S235JR,MATERIAL IS OILED,MATERIAL IS OILED,0.00,0.0,5302.0,12.00,0.0,0.0,0.00,0.0000,no
98,23043348.0,DX51D +Z140,MATERIAL IS OILED,MATERIAL IS OILED,0.00,0.0,16927.0,69.00,0.0,0.0,0.00,0.0000,no
